# Hypothesis Testing 2 - Examining Racial Discrimination in the US Job Market

Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

Let's perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does our analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how should we amend our analysis?


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states

In [108]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [9]:
# number of callbacks for white-sounding names
sum(data[data['race'] == 'w']['call'])

235.0

In [10]:
# number of callbacks for black-sounding names
sum(data[data['race'] == 'b']['call'])

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## 1. What test is appropriate for this problem? Does CLT apply?

In [4]:
w = data[data['race'] == 'w']
b = data[data['race'] == 'b']

In [82]:
w_call = len(w[w['call'] == True])
w_nocall = len(w[w['call'] == False])

In [87]:
b_call = len(b[b['call'] == True])
b_nocall = len(b[b['call'] == False])

In [89]:
print('For the white-sounding names group, we have %.0f callbacks and %.0f without callbacks.' % (w_call, w_nocall))

For the white-sounding names group, we have 235 callbacks and 2200 without callbacks.


In [90]:
print('For the black-sounding names group, we have %.0f callbacks and %.0f without callbacks.' % (b_call, b_nocall))

For the black-sounding names group, we have 157 callbacks and 2278 without callbacks.


Two sample two tailed z test is appropriate for this problem as we are interested in whether there is a difference in call back rate between the two independent samples.

Let's check for CLT Assumptions:
- Random sample: Our samples are randomly assigned to the white-sounding and black-sounding group.
- Large sample: We have more than 30 callbacks and non-callbacks for both white-sounding and black-sounding group. Thus, so our assumption of large enough sample size is valid.
- Independent sample: Assuming the population is all job seekers in the U.S., our sample is less than 10% of the population and so the assumption of independent sample is valid.

Since the CLT assumptions are valid, we can apply CLT.

## 2. Null and Alternative Hypotheses

Null Hypothesis: call back rate for white-sounding group - call back rate for black-sounding group = 0

Alternative Hypothesis: call back rate for white-sounding group - call back rate for black-sounding group =/= 0

## 3. Bootstrapping and Frequentist hypothesis testing

### Bootstrap approach

#### Margin of Error

In [91]:
def draw_bs_reps(data1, data2, size=1):

    bs_replicates = np.empty(size)

    for i in range(size):
        prop1 = np.mean(np.random.choice(data1, size=len(data1)))
        prop2 = np.mean(np.random.choice(data2, size=len(data2)))
        bs_replicates[i] = prop1 - prop2

    return bs_replicates

In [92]:
bs_replicates = draw_bs_reps(w['call'], b['call'], size=10000)
margin_error = 1.96 * np.std(bs_replicates)

In [93]:
print('Assuming a significance level of 0.05, the margin of error is %.3f' % (margin_error))

Assuming a significance level of 0.05, the margin of error is 0.015


#### Confidence Interval

In [94]:
[lower, upper] = np.percentile(bs_replicates, [2.5, 97.5])

In [95]:
print('The 95 percent confidence interval is between %.3f and %.3f' % (lower, upper))

The 95 percent confidence interval is between 0.017 and 0.048


#### P value

In [98]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [99]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

In [100]:
def diff_white_black(white, black):
    diff = np.mean(white) - np.mean(black)
    return diff

In [101]:
perm_replicates = draw_perm_reps(w['call'], b['call'], diff_white_black, 10000)

In [102]:
p_value = np.sum(perm_replicates >= (np.mean(w['call']) - np.mean(b['call']))) / len(perm_replicates)
print('p_value =', p_value)

p_value = 0.0


### Frequentist approach

#### Margin of Error

In [103]:
pw = np.mean(w['call'])
pb = np.mean(b['call'])

In [104]:
margin_error = 1.96 * ((pw * (1 - pw) / 2435) + (pb * (1 - pb) / 2435)) ** 0.5

In [105]:
print('Assuming a significance level of 0.05, the margin of error is %.3f' % (margin_error))

Assuming a significance level of 0.05, the margin of error is 0.015


#### Confidence Interval

In [106]:
lower = (pw - pb) - margin_error
upper = (pw - pb) + margin_error

In [107]:
print('The 95 percent confidence interval is between %.3f and %.3f' % (lower, upper))

The 95 percent confidence interval is between 0.017 and 0.047


#### P value

In [109]:
z_stat, p_value = proportions_ztest([235, 157], [2435, 2435], value=0, alternative='two-sided')

In [112]:
print('Statistics = %.3f, p_value = %.3f' % (z_stat, p_value))

Statistics = 4.108, p_value = 0.000


## 4. Conclusion

Based on our samples, we reject the null hypothesis that the call back rate between the white-sounding group and the black-sounding group are the same and conclude that race has a significant impact on the rate of callbacks for resumes using a significance level of 0.05. We are also 95% confident that the difference between the call back rate for white-sounding vs black-sounding group lies between 0.017 and 0.042, which means the call back rate for white-sounding group is higher than that for black-sounding group.

## 5. Is race/name is the most important factor in callback success?

My analysis only proves that race has a significant impact on callback success but that doesn't mean that race is the most important factor. To find out what is the most important factor we will need to build a classification machine learning model with call back success as the outcome and different factors including race as features for predicting outcome. We can build a Logistic Regression model and compare the weights of different features and the most important feature would be the one with the highest absolute value of weight. We can also build a Random Forest model and look at the feature importance score to find out the most important feature.